In [6]:
import time
from selenium import webdriver
import csv
import urllib.request as req
import random
from datetime import datetime
import pathlib
from selenium.webdriver.common.by import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import pandas as pd


In [7]:
options = webdriver.ChromeOptions() 
script_directory = pathlib.Path().absolute()
options.add_argument(f"--user-data-dir={script_directory}\\chrome_data")
driver = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.
driver.get('https://thatsthem.com/login')


In [12]:
def regex_with_finditer(regex,str_r,index=False):
    import re
    r_out=[]
    matches = re.finditer(regex, str_r, re.MULTILINE)

    for matchNum, match in enumerate(matches, start=1):
        if index:
            temp=[match.group(), match.start(),match.end()]
            r_out.append(temp)
        else:
            r_out.append(match.group())
        
    return r_out



In [13]:
def search(email):
    driver.execute_script('document.querySelector("#searchbar").value=""')
    driver.find_element(By.CSS_SELECTOR, "#searchbar").send_keys(email)
    driver.find_element(By.CSS_SELECTOR,'#searchform label[for="email"][class="radiobutton_width"]').click()
    
    driver.execute_script('''document.querySelector('#searchform input[value="Search"][class="form_submit"][type="submit"]').click()''')
def count_substring_matches(string, substring):
    count = 0
    start = 0

    while True:
        index = string.find(substring, start)
        if index == -1:
            break
        count += 1
        start = index + 1

    return count
def return_highligher(single_data,email):
        badge=''
        for i in single_data:
            if i['email']==email:
                for h in highlighter_databases:
                    if h[0]==i['database']:
                        badge+=f' ({h[1]})'
        return badge
                


In [14]:

headers =  ['email','database','address','password','ip','username','name','phone','birthday','hash']
excel_headers=['username','email','password']
with open('export.csv', 'w',encoding="utf-8", newline='') as file:
    writer = csv.DictWriter(file, fieldnames=excel_headers)
    writer.writeheader()
# write a export file from before



f = open("import.txt", "r",encoding='utf-8')

for x in f:
    pws=[]
    single_data=[]
    keyword=x.replace('\n','')

    emails=keyword.split(':')[1].split(',')
    export_file=keyword.split(':')[0].replace('@','')
    # 1. separate email and output file
    for email in emails:
        search(email)
        # 2. search first

        time.sleep(3)
        ok=False


        if not (len(driver.find_elements(By.CSS_SELECTOR,'#scrollArea #contentArea > div , #combolists > div')) > 60):
            for i in driver.find_elements(By.CSS_SELECTOR,'#scrollArea #contentArea > div a[onclick*="full("].searchtools_button'):
                try:
                    i.click()
                    time.sleep(0.5)
                except:
                    pass
            # 3. search secondpass
            # 3.click on view full

            for i in driver.find_elements(By.CSS_SELECTOR,'#scrollArea #contentArea > div .xhash'):
                try:
                    i.click()
                    time.sleep(0.5)
                except:
                    pass
            # 4.click on unhashing 
            
            for i in driver.find_elements(By.CSS_SELECTOR,'#scrollArea #contentArea > div .xlastip'):
                try:
                    i.click()
                    time.sleep(0.5)
                except:
                    pass
            # 5.click on lastip 
            time.sleep(10)
            for i in driver.find_elements(By.CSS_SELECTOR,'#scrollArea #contentArea > div '):
                data={'email':'','database':'','address':'','password':'','ip':'','username':'','name':'','phone':'','birthday':'','hash':''}
                # 6.predefined variables

                data['database']=i.find_element(By.CSS_SELECTOR,'#topBar').text.replace('Loading...','')
                # 7.get database name

                for tr in i.find_elements(By.CSS_SELECTOR,'tr'):
                    key=tr.find_elements(By.CSS_SELECTOR,'td')[0].text.lower()
                    value=tr.find_elements(By.CSS_SELECTOR,'td')[1].text
                    if(key=='email'):
                        data['email']=value
                    elif(key=='username'):
                        data['username']=value
                    elif('name' in key):
                        data['name']+=value
                    elif('phone' in key):
                        data['phone']=value
                    elif('birthday' in key):
                        data['birthday']=value
                    elif('hash' in key):
                        data['hash']=value.split('\n')[0]
                        try:
                            data['password']=value.split('\n')[1]
                        except:
                            pass
                    elif('password' in key):
                        data['password']=value
                    elif('lastip' in key):
                        data['ip']=value.split('\n')[0]
                        try:
                            data['address']+=' '+value.split('\n')[1]
                        except:
                            pass
                    
                    else:
                        location_substrings=['country' , 'address' , 'address1' , 'street1' , 'address2' , 'addr1' , 'addr2' , 'city' , 'state' , 'country' , 'zip' , 'province' , 'province_code' , 'country_code' , 'zipcode' , 'postcode' , 'fu_address' , 'fu_city' , 'fu_country' , 'fu_zip' , 'area' , 'Ip_country' , 'zipcode' , 'fu_zip' , 'province_code' , 'postal' , 'postalcode']
                        
                        for location in location_substrings:
                            if location in key:
                                data['address']+=' '+value
                                break
                    # 8.extract information

                
                with open('export.csv', 'a',encoding="utf-8", newline='') as file:
                    writer = csv.DictWriter(file, fieldnames=excel_headers)
                    if not len(data['password'])>20 and data['password'].strip().rstrip() and data['password'] not in pws and not '.' in  data['password']:
                        writer.writerow({'username':export_file,'email':data['email'],'password':data['password']})
                        pws.append(data['password'])
                    if len(data['password'])>20:
                        data['password'] = ''
                    single_data.append(data)
            # 9.expoer ro csv  

            


            for i in driver.find_elements(By.CSS_SELECTOR,'#combolists > div'):
                try:
                    i.find_element(By.CSS_SELECTOR,'.expand').click()
                except:
                    pass
                dbname=i.find_element(By.CSS_SELECTOR,'.topbar').text.split('-')[0]
            
                for ii in i.find_elements(By.CSS_SELECTOR,'.combolist_res > div'):
                    data={'email':'','database':'','address':'','password':'','ip':'','username':'','name':'','phone':'','birthday':'','hash':''}
                    email=ii.get_attribute('innerHTML').split('<span class="combo_sep"> </span>')[0]
                    pwd=ii.get_attribute('innerHTML').split('<span class="combo_sep"> </span>')[1]
                    data['email']=email
                    data['password']=pwd
                    with open('export.csv', 'a',encoding="utf-8", newline='') as file:
                        writer = csv.DictWriter(file, fieldnames=excel_headers)
                        if not len(pwd)>20 and pwd.strip().rstrip() and data['password'] not in pws and not '.' in  data['password']:
                            writer.writerow({'username':export_file,'email':data['email'],'password':data['password']})
                            pws.append(data['password'])
                        if len(pwd)>20:
                            data['password'] = ''
                        single_data.append(data)
                # 6.predefined variables
            print(email+' Done')

        else:
            print(email+' have More then 60 results . so skipping.....')
          


    
        

    txt=''
    for i in single_data:
        for ii in i:
            if "database" in ii:
                continue
            txt+=ii+'\n'
            r=0
            for i2 in single_data:
                if i2[ii].lower() not in txt.lower() and i2[ii].strip().rstrip().lower():
                    txt+=i2[ii].strip().rstrip()+f' ({count_substring_matches(str(single_data).lower(),i2[ii].lower())} Results) '+return_highligher(single_data,i2[ii])+'\n'
                    r+=1
            txt+='\n\n\n'
        break
    f = open('./data/'+export_file+".txt", "w", encoding="utf-8")
    f.write(txt)
    f.close()
   

file_name_output = "export.xlsx"

df = pd.read_excel(file_name_output) 



# Step 2: Drop duplicate rows based on lowercase columns
df = df.drop_duplicates()

# Optionally, Step 3: Revert the columns back to their original case





# Write the results to a different file
df.to_excel(file_name_output, index=False)
   

thetruelegitboss@gmail.com Done
kevinwade74@yahoo.com Done
hdaeae@hotmail.com Done
jcllafleur@yahoo.com Done
maikel_coeleman@hotmail.com Done
maikel_coeleman@hotmail.com Done
samuelnickey@yahoo.com Done
paglia92-@hotmail.it Done
cotd@hotmail.com Done
athleticsnumber44@gmail.com Done
Cabbage363@hotmail.com Done
Omfg_itz_harry@hotmail.com Done
funnyjj12312@gmail.com Done
inteproth@aol.com Done
i_am_china_timo@hotmail.de Done
markdavies637@gmai.com Done
ironmortel85@hotmail.com Done
koeskoes2041@mail.com Done
niels_1990@hotmail.com Done
koolio303@gmail.com Done
kool12h@hotmail.com Done
koolio303@gmail.com Done
koolio202@gmail.com Done
koolio303@gmail.com Done
koolio202@gmail.com Done
william_joseph@hotmail.com Done
michaelmarino8@msn.com Done
michaelmarino8@msn.com Done
michaelmarino8@msn.com Done
michaelmarino8@msn.com Done
michaelmarino8@msn.com Done
lucascarpino@hotmail.com Done
lucascarpino@hotmail.com Done
lucascarpino@hotmail.com Done
lucascarpino@hotmail.com Done
true2dastreets@hot

JavascriptException: Message: javascript error: Cannot set properties of null (setting 'value')
  (Session info: chrome=115.0.5790.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00CDA813+48355]
	(No symbol) [0x00C6C4B1]
	(No symbol) [0x00B75358]
	(No symbol) [0x00B787A1]
	(No symbol) [0x00B7A19B]
	(No symbol) [0x00BCD279]
	(No symbol) [0x00BBA73C]
	(No symbol) [0x00BCC922]
	(No symbol) [0x00BBA536]
	(No symbol) [0x00B982DC]
	(No symbol) [0x00B993DD]
	GetHandleVerifier [0x00F3AABD+2539405]
	GetHandleVerifier [0x00F7A78F+2800735]
	GetHandleVerifier [0x00F7456C+2775612]
	GetHandleVerifier [0x00D651E0+616112]
	(No symbol) [0x00C75F8C]
	(No symbol) [0x00C72328]
	(No symbol) [0x00C7240B]
	(No symbol) [0x00C64FF7]
	BaseThreadInitThunk [0x76FD00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77737B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77737AEE+238]


In [ ]:
single_data

[{'email': 'ceirbus@yahoo.com',
  'database': 'TWITTER_COM_228M_SOCIAL_012023',
  'address': '',
  'password': '',
  'ip': '',
  'username': 'MrCeirbus',
  'name': 'Will Faircloth',
  'phone': '',
  'birthday': '',
  'hash': ''},
 {'email': 'ceirbus@yahoo.com',
  'database': 'DATPIFF_COM_7M_MUSIC_2021',
  'address': '',
  'password': '',
  'ip': '',
  'username': 'ceirbus',
  'name': '',
  'phone': '',
  'birthday': '',
  'hash': '7d18e18f502d7662991aa06eb517052e'},
 {'email': 'ceirbus@yahoo.com',
  'database': 'ZYBEZ_NET_151K_GAMING_022016',
  'address': ' Charlotte, North Carolina, United States',
  'password': '',
  'ip': '71.68.107.56',
  'username': 'williscoolerthanu',
  'name': '',
  'phone': '',
  'birthday': '',
  'hash': '4c91eebf0129bbfe214b4f13cfec160e'},
 {'email': 'ceirbus@yahoo.com',
  'database': 'MYSPACE_COM_357M_SOCIAL_2008',
  'address': '',
  'password': 'eragon77',
  'ip': '',
  'username': 'screamoroxmysox',
  'name': '',
  'phone': '',
  'birthday': '',
  'hash':